In [3]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
!pip install import_ipynb
import import_ipynb
import model_test_for_openmax_add_CDF

In [4]:
input_data = input()         # 아직 입력부를 구현하지 않아 임시로 입력

안녕하세요 나는 진하빈입니다


In [5]:
def tokenize(x_data, tokenizer):
    tokenizer.fit_on_texts(x_data) # 데이터의 각 행별로 토큰화 수행
    return tokenizer.texts_to_sequences(x_data)    

In [50]:
x_data = input_data
y_data = 0    # 임시 값

In [51]:
tokenizer = Tokenizer()
sequences = tokenize(x_data, tokenizer)

## 모델 적용

In [52]:
!pip install keras.models
from keras.models import model_from_json
json_file = open("model.json", "r")
loaded_model_json = json_file.read() 
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [60]:
loaded_model.load_weights("model_weight.h5")

In [70]:
#model.compile(optimizer='adam', loss=SparseCategoricalCrossentropy(from_logits=True),metrics= ["accuracy"])
#es = EarlyStopping(monitor='loss', mode='min' , min_delta=0)
#check_point = ModelCheckpoint('apply_model.h5', monitor='loss', mode='min', save_best_only=True)
loaded_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics= ["accuracy"])

In [76]:
X_result = loaded_model.evaluate(x_data, y_data)

IndexError: list index out of range

## logit Vector, Distance 계산

In [36]:
class_result = 0
class_result = X_result.index(max(X_result))

tf_result = (y_test == class_result)

if(tf_result == True):
    if(class_result == 0):
        for_open_max_0 = X_result
    if(class_result == 1):
        for_open_max_1 = X_result
    if(class_result == 2):
        for_open_max_2 = X_result

AttributeError: 'NoneType' object has no attribute 'index'

In [ ]:
input_average_0 = average_vector(for_open_max_0, class_result) #평균 Logit Vector - Class 0
input_average_1 = average_vector(for_open_max_1, class_result) #평균 Logit Vector - Class 1
input_average_2 = average_vector(for_open_max_2, class_result) #평균 Logit Vector - Class 2

In [ ]:
dist0 = distance(for_open_max_0, input_average_0)
dist1 = distance(for_open_max_1, input_average_1)
dist2 = distance(for_open_max_2, input_average_2)

## CDF 계산

In [ ]:
def calculCDF(dist):
    CDF = []
    for i in dist:
        CDF.append(s.exponweib.cdf(i, *s.exponweib.fit(i, 1, 1, scale=2, loc=0)))     
        return CDF
CDF0 = calculCDF(dist0)
CDF1 = calculCDF(dist1)
CDF2 = calculCDF(dist2)

## Logit Vetcr 업데이트

In [ ]:
updated_logit = []
i = 0

logit0 = X_result[0]-(X_result[0]*CDF0[i])
logit1 = X_result[1]-(X_result[1]*CDF1[i])
logit2 = X_result[2]-(X_result[2]*CDF2[i])
unkn_logit = CDF0[i]*X_result[0] + CDF1[i]*X_result[1] + CDF2[i]*X_result[2]    # unknown class의 logit vector
updated_logit.append([unkn_logit, logit0, logit1, logit2])     
i += 1

In [ ]:
# update 된 logit veoctor로 softmax layer 통과

## Softmax Layer 통과

In [ ]:
new_model = Sequential()
new_model.add(Dense(num_of_class + 1, activation = 'softmax'))
optimizer='adam', loss='sparse_categorical_crossentropy',metrics=["accuracy",f1,precision, recall])
check_point = ModelCheckpoint('new_best_model.h5', monitor='loss', mode='min', save_best_only=True)

In [ ]:
new_model.fit(x_data, y_data, batch_size = 16, epochs=10, callbacks=[es, check_point])

In [ ]:
new_model.predict(X_result, batch_size = 16)

## 결과 확인

In [ ]:
result["Classification Result"] = class_result
result["Final Result"] = (y_test == class_result)

In [ ]:
print("테스트 정확도: %.4f" % (model.evaluate(x_data, y_data)[1]))

## 모델 저장

In [ ]:
##모델을 .json 파일 형식으로 save하여 저장
model_json = model.to_json()
with open("new_model.json", "w") as json_file : 
    json_file.write(model_json)
    
model.save_weights("new_model_weight.h5")
model.save('new_full_model.h5')